In [19]:
import os
import sys
import random
import urllib.parse
import time
import re

import markovify
import json
from bs4 import BeautifulSoup
import spacy

sys.path.insert(0, os.path.abspath('..'))

import ao3
from ao3.utils import work_id_from_url
from ao3.works import Work

In [2]:
def add_work_to_chain(work, models, state_size=3):
    '''adds new Markov chain model of work
    with given ID to the dict models and returns said dict'''
    text = work.get_work_text()
    model = markovify.Text(text, state_size=state_size)
    models[str(work.id)] = model
    print('Added work {} to models.'.format(work.title))

    return models

In [3]:
def load_data():
    with open('data.txt', 'r') as f:
        raw_data = f.readlines()

    bookmark_data = {}
    in_bookmark = False
    ending_bookmark = False
    current_id = '3413384'
    current_data = ''
    for line in raw_data:
        if ending_bookmark:
            # print('Bookmark is ending')
            ending_bookmark = False
            parts = line.partition('>')
            current_data += parts[0]
            current_data += parts[1]
            # print('Partitioned line: {}'.format(parts))
            bookmark_data[current_id] = current_data # add tag to dict
            # print('Assigned current data to id {}'.format(current_id))
            current_id = parts[2].strip() # assign next id
            # print('Assigned next id as {}'.format(current_id))
            current_data = ''
        else:
            current_data += line
            if '<div class="recent dynamic"' in line:
                ending_bookmark = True
                # print('Turned on ending bookmark')
    
    print('Logging in...')
    api.login('starrybouquet')
    all_works = []
    for id, html in bookmark_data.items():
        soup = BeautifulSoup(html, 'html.parser')
        all_works.append(Work(id, api.handler, load=False, soup=soup))
        
    possible_tags = set(['Samantha "Sam" Carter/Jack O\'Neill', 'Jack/Sam', 'Sam/Jack', 'Sam Carter/Jack O\'Neill'])
    matching_works = []
    for work in all_works:
        ship = set(work.relationship)
        intersection = possible_tags.intersection(ship)
        if len(intersection) > 0:
            matching_works.append(work)
    print('Works matching tags: ')
    print(len(matching_works))
    return matching_works

In [4]:
def create_models(works):
    models = {} # each entry is id: model
    ids = []
    
    models_built = 1
    for work in works:
        print('Building model {}...'.format(models_built))
        if models_built % 40 == 0:
            print('{} models built, sleeping 3 min'.format(models_built))
            time.sleep(180)
        models = add_work_to_chain(work, models)
        models_built += 1
    print('Done building models.')
    print('keys of model dict:')
    print(models.keys())
    return models

In [5]:
api = ao3.AO3()
works = load_data()
input('Press Enter to continue: ')
models = create_models(works)

Logging in...
Authentication successful. You are logged in.
Works matching tags: 
179
Building model 1...
Added work Auctions and Consequences to models.
Building model 2...
Added work Better or Right to models.
Building model 3...
Added work dark switch to models.
Building model 4...
Added work level three, advanced to models.
Building model 5...
Added work If the World was Ending to models.
Building model 6...
Added work Walking the Dog to models.
Building model 7...
Added work a shine of light and a circle of blue to models.
Building model 8...
Added work F*@kingPete to models.
Building model 9...
Added work T-minus a Lifetime to models.
Building model 10...
Added work Walking on Broken Glass to models.
Building model 11...
Added work Singing to models.
Building model 12...
Added work Despair and hope to models.
Building model 13...
Added work Make it rain to models.
Building model 14...
Added work Revelations to models.
Building model 15...
Added work Solstitium, Interrupted to mod

In [6]:
json_models = {}
models_list = []
for workid, model in models.items():
    json_model = model.to_json()
    json_models[workid] = json_model

In [8]:
with open('work_models.json', 'w') as file:
    json.dump(json_models,file)

In [9]:

# load
with open('work_models.json', 'r') as f:
    raw = f.read()
print(raw[:1000])
models_from_json = json.loads(raw)
modelList = []
models = {}
for workid, model_json in models_from_json.items():
    actual_model = markovify.Text.from_json(model_json)
    modelList.append(actual_model)
    models[str(workid)] = actual_model
print('Models loaded')



{"3413384": "{\"state_size\": 3, \"chain\": \"[[[\\\"___BEGIN__\\\", \\\"___BEGIN__\\\", \\\"___BEGIN__\\\"], {\\\"Colonel\\\": 1, \\\"Daniel\\\": 12, \\\"Their\\\": 1, \\\"Putting\\\": 1, \\\"The\\\": 12, \\\"There\\\": 8, \\\"No\\\": 2, \\\"This\\\": 5, \\\"Then\\\": 1, \\\"Not\\\": 5, \\\"And\\\": 3, \\\"Now,\\\": 1, \\\"Teal'c's\\\": 1, \\\"With\\\": 3, \\\"Or,\\\": 2, \\\"I'm\\\": 3, \\\"However,\\\": 1, \\\"Jack\\\": 34, \\\"So\\\": 2, \\\"Teal'c\\\": 7, \\\"Considering\\\": 3, \\\"He\\\": 29, \\\"Daniel,\\\": 1, \\\"Why\\\": 2, \\\"No,\\\": 2, \\\"Still,\\\": 2, \\\"A\\\": 4, \\\"From\\\": 3, \\\"They'll\\\": 1, \\\"We\\\": 3, \\\"They\\\": 3, \\\"As\\\": 4, \\\"If\\\": 1, \\\"Killing\\\": 1, \\\"They're\\\": 3, \\\"I\\\": 2, \\\"Don't\\\": 1, \\\"Yeah,\\\": 1, \\\"It\\\": 6, \\\"Disgusted,\\\": 1, \\\"Her\\\": 12, \\\"Boots\\\": 1, \\\"Plus,\\\": 1, \\\"After\\\": 1, \\\"Carter\\\": 12, \\\"She\\\": 22, \\\"Almost\\\": 1, \\\"On\\\": 1, \\\"Wary\\\": 1, \\\"Getting\\\": 1, \\\"

In [14]:
def combine_models(models, weights=None):
    if not weights:
        model = markovify.combine(models)
    else:
        model = markovify.combine(models, weights)
    return model.compile()

In [15]:

# combine
full_model = combine_models(modelList)
num_sentences = int(input('Enter # of sentences to try: '))
for i in range(num_sentences):
    print(full_model.make_sentence())

save = str(input('Do you want to save this model? y or n: '))
if save == 'y':
    filename = str(input('Enter filename to save: '))
    json_model = full_model.to_json()
    with open(filename, 'w') as f:
        json.dump(json_model, f)
    print('Saved file.')

Or rather, one side of his neck, and her fingers trace a slow, deliberate caress from his shoulder blade to his waist, and his fingers pressing at her entrance and then he goes on talking about a deceased loved one.
He calls her cell phone and fished around in her mind.
None
Their eyes met, and this time held up a hand when he made to speak.
What to say to her.
Five minutes later she’s in her lab, hoping she’d get to find one of them.
And when she’d told him that maybe she was in his arms and wrapped herself so tightly around his hips, pressing him closer and he tears at the material instead.
Jack bit back a smile.
“You know,” he shrugged as he took a couple of times, but he doesn’t speak until he climbs the ramp to stand at the door.“What?” he grins.
She always knew what to say or at least one of their silent conversations.
She thinks she slept for a while, they were happy.
Someone was going to pull the hose off, after which he repeated the process she had done just that.
The music fa

NameError: name 'ids' is not defined

In [16]:
num_sentences = int(input('Enter # of sentences to try: '))
for i in range(num_sentences):
    print(full_model.make_sentence())

Instead, he stood his ground and closed the door and walking past him to work on the other side of a pillar several hundred feet away.
Her eyes lit up with an explanation until finally it dawned her exactly what Jack was trying to hide a smile behind her hand.“You’re laughing at me.”She turned and wrapped her arms around her body to relax.“I know,” he says, stroking her back.
*Day 1, One Week Earlier* “Carter, I swear to God if I have to get them any higher.
I thought it was super glue?” he protested, yet still careful to keep the truth from themselves than it was in those intimate, stolen moments that Jacob saw a side to the two pieces popped back where they belonged.At least they went back where they belonged.At least they went back where they belonged.
The scent of him that didn’t want her to dwell on the possibility of them.
She leaned next to him on the couch, folding her leg underneath her and facing him.
There are the usual flowers and seating plans in the lobby but none of it h

In [21]:
nlp = spacy.load("en_core_web_sm")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [22]:
# load
with open('work_models.json', 'r') as f:
    raw = f.read()
models_from_json = json.loads(raw)
modelList = []
models = {}
for workid, model_json in models_from_json.items():
    actual_model = POSifiedText.from_json(model_json)
    modelList.append(actual_model)
    models[str(workid)] = actual_model
print('Models loaded')


# combine
full_model = combine_models(modelList)
num_sentences = int(input('Enter # of sentences to try: '))
for i in range(num_sentences):
    print(full_model.make_sentence())

save = str(input('Do you want to save this model? y or n: '))
if save == 'y':
    filename = str(input('Enter filename to save: '))
    json_model = full_model.to_json()
    with open(filename, 'w') as f:
        json.dump(json_model, f)
    print('Saved file.')

{"3413384": "{\"state_size\": 3, \"chain\": \"[[[\\\"___BEGIN__\\\", \\\"___BEGIN__\\\", \\\"___BEGIN__\\\"], {\\\"Colonel\\\": 1, \\\"Daniel\\\": 12, \\\"Their\\\": 1, \\\"Putting\\\": 1, \\\"The\\\": 12, \\\"There\\\": 8, \\\"No\\\": 2, \\\"This\\\": 5, \\\"Then\\\": 1, \\\"Not\\\": 5, \\\"And\\\": 3, \\\"Now,\\\": 1, \\\"Teal'c's\\\": 1, \\\"With\\\": 3, \\\"Or,\\\": 2, \\\"I'm\\\": 3, \\\"However,\\\": 1, \\\"Jack\\\": 34, \\\"So\\\": 2, \\\"Teal'c\\\": 7, \\\"Considering\\\": 3, \\\"He\\\": 29, \\\"Daniel,\\\": 1, \\\"Why\\\": 2, \\\"No,\\\": 2, \\\"Still,\\\": 2, \\\"A\\\": 4, \\\"From\\\": 3, \\\"They'll\\\": 1, \\\"We\\\": 3, \\\"They\\\": 3, \\\"As\\\": 4, \\\"If\\\": 1, \\\"Killing\\\": 1, \\\"They're\\\": 3, \\\"I\\\": 2, \\\"Don't\\\": 1, \\\"Yeah,\\\": 1, \\\"It\\\": 6, \\\"Disgusted,\\\": 1, \\\"Her\\\": 12, \\\"Boots\\\": 1, \\\"Plus,\\\": 1, \\\"After\\\": 1, \\\"Carter\\\": 12, \\\"She\\\": 22, \\\"Almost\\\": 1, \\\"On\\\": 1, \\\"Wary\\\": 1, \\\"Getting\\\": 1, \\\"

In [23]:
num_sentences = int(input('Enter # of sentences to try: '))
for i in range(num_sentences):
    print(full_model.make_sentence())

But he hasn’t properly slept in about thirty-six hours and, while it’s not the weirdest thing we’ve come across but…” She trails off.
“You also aren’t supposed to have a firm grip on the man’s hand tighten just enough to get access to her.
“Are we really going to not say it?”“I don’t think we should do something really normal,” Jack says.
If this was how they looked in the fridge with a sigh.
Colonel O'Neill yells, as he and Teal'c position themselves to cover the rest of his injuries had somehow helped accelerate the healing of the concussion.
Sam reached up to the cabin much sooner than she had.
She straightens her shoulders, hands looking for a point mutation and see if the stuff broke down on its own.
He had a very bad feeling about today, but tried to shrug it off.
For the moment, all he cared about was her body slamming into the passenger side dash.
Together they shuffled to the edge and hovered her hands over her backpack.
Walter started passing out the folders he held in his ar